# Building a live tuner

We will now build a live tuner using the information that we have seen so far.

However there is one part of the program that we are not familiar with yet:<br>
*The direct input of audio*

## Step 1: Get a live-stream (in - out)
Obviously we will need to be able to record from our computer's default sound input device and output the sound simultaniously using the computer's default sound output device.

To do this we will use the [`python-sounddevice`](https://python-sounddevice.readthedocs.io/en/0.4.1/) module, and more specifically the [Stream](https://python-sounddevice.readthedocs.io/en/0.4.1/api/streams.html?highlight=Stream#streams-using-numpy-arrays) methods.

> *sources:*
> - *[youtube: Audio spectrum analyszer](https://www.youtube.com/watch?v=AShHJdSIxkY)*
> - *[Speeding up Matplotlib](https://bastibe.de/2013-05-30-speeding-up-matplotlib.html)*

In [11]:
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt

%matplotlib tk

blocksize = 1024 * 4

In [2]:
def playAudio(in_data, out_data, frames, time, status):
    """Callback function of the getAudio() functions
    """
    out_data[:] = in_data    
    
def getAudio(sr=44100):
    """Access Audio input
    """
    with sd.Stream(
        samplerate=sr,
        blocksize=blocksize,
        device=None,
        channels=None,
        dtype=None,
        latency=None,
        extra_settings=None,
        callback=playAudio,
        finished_callback=None,
        clip_off=None,
        dither_off=None,
        never_drop_input=None,
        prime_output_buffers_using_stream_callback=None
    ):
        print("Press 'Enter' key to stop")
        input()

In [3]:
try:
    getAudio()
except KeyboardInterrupt:
    parser.exit('')

Press 'Enter' key to stop


## Step 2: Get a live graph

The main problem that we have with graphing live audio is that matplotlib is plotting a fixed array of values. This means that we have multiple solutions at our disposal:
1. Creating a graph with a buffer-size of n. Destroy it and recreating it with the next buffer
2. Creating a graph with n items and updating the graph for each buffer
3. Creating a graph with n items and for each piece of input information, slide the nth item to `itemList[n-1]` replacing the value of the nth item with `n+1`.

To do any of these methods we need a buffer, or a queue.

Luckily, python has a built-in module called `queue`. Its main class is the `queue.Queue()` class and allows us to create a simple FIFO quye type.

Now have a look at the Tuner directory to see how it is made.

Please note how in the main function (graphAudio), the callback for the Audio Stream is only creating an array of data and nothing else.

It is a separate Animation function called [`FuncAnimation()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html) that updates the matplotlib graph.